In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

Dataset Path

In [8]:
train_dir = r'E:\Project_UMER\ASL_detection\ASL_detection\dataset\asl_alphabet_train'
test_dir = r'E:\Project_UMER\ASL_detection\ASL_detection\dataset\asl_alphabet_test'

Image Parameters

In [4]:
IMG_HEIGHT, IMG_WIDTH = 128, 128
BATCH_SIZE = 32

Augmentation & data loading

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    validation_split=0.2 
)

In [9]:
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

Found 69600 images belonging to 29 classes.


In [10]:
val_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 17400 images belonging to 29 classes.


In [11]:
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 0 images belonging to 0 classes.


Basic CNN Model

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [13]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(29, activation='softmax')  
])

d:\mlops\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Training the model

In [16]:
EPOCHS = 10

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS
)

Epoch 1/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 722s 332ms/step - accuracy: 0.3529 - loss: 2.0761 - val_accuracy: 0.6039 - val_loss: 1.1369
Epoch 2/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 705s 324ms/step - accuracy: 0.7210 - loss: 0.8085 - val_accuracy: 0.7105 - val_loss: 0.8870
Epoch 3/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 712s 327ms/step - accuracy: 0.8188 - loss: 0.5227 - val_accuracy: 0.7652 - val_loss: 0.7077
Epoch 4/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 701s 322ms/step - accuracy: 0.8618 - loss: 0.3934 - val_accuracy: 0.7717 - val_loss: 0.6822
Epoch 5/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 696s 320ms/step - accuracy: 0.8952 - loss: 0.3075 - val_accuracy: 0.8044 - val_loss: 0.6502
Epoch 6/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 721s 331ms/step - accuracy: 0.9119 - loss: 0.2575 - val_accuracy: 0.8201 - val_loss: 0.5965
Epoch 7/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 706s 325ms/step - accuracy: 0.9223 - loss: 0.2324 - val_accuracy: 0.8135 - val_loss: 0.6037
Epoch 8/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 665s 306ms/step - ac

Saving the model

In [19]:
model.save("asl_model.keras")

Evaluation on the test set

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Debug

In [25]:
import os

if os.path.exists(test_dir):
    print("Test directory exists.")
    subfolders = os.listdir(test_dir)
    print("Subfolders (classes):", subfolders)
    for folder in subfolders:
        folder_path = os.path.join(test_dir, folder)
        if os.path.isdir(folder_path):
            print(f"Class '{folder}' contains {len(os.listdir(folder_path))} images.")
        else:
            print(f"'{folder}' is not a directory.")
else:
    print("Test directory not found!")

Test directory exists.
Subfolders (classes): ['A_test.jpg', 'B_test.jpg', 'C_test.jpg', 'D_test.jpg', 'E_test.jpg', 'F_test.jpg', 'G_test.jpg', 'H_test.jpg', 'I_test.jpg', 'J_test.jpg', 'K_test.jpg', 'L_test.jpg', 'M_test.jpg', 'nothing_test.jpg', 'N_test.jpg', 'O_test.jpg', 'P_test.jpg', 'Q_test.jpg', 'R_test.jpg', 'space_test.jpg', 'S_test.jpg', 'T_test.jpg', 'U_test.jpg', 'V_test.jpg', 'W_test.jpg', 'X_test.jpg', 'Y_test.jpg', 'Z_test.jpg']
'A_test.jpg' is not a directory.
'B_test.jpg' is not a directory.
'C_test.jpg' is not a directory.
'D_test.jpg' is not a directory.
'E_test.jpg' is not a directory.
'F_test.jpg' is not a directory.
'G_test.jpg' is not a directory.
'H_test.jpg' is not a directory.
'I_test.jpg' is not a directory.
'J_test.jpg' is not a directory.
'K_test.jpg' is not a directory.
'L_test.jpg' is not a directory.
'M_test.jpg' is not a directory.
'nothing_test.jpg' is not a directory.
'N_test.jpg' is not a directory.
'O_test.jpg' is not a directory.
'P_test.jpg' is no

Deployment

In [21]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

Loading the model

In [22]:
model = load_model("asl_model.h5")

Prediction Function

In [23]:
def predict_sign(image_path):
    image = load_img(image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    image_array = img_to_array(image) / 255.0
    image_array = np.expand_dims(image_array, axis=0)
    
    predictions = model.predict(image_array)
    class_idx = np.argmax(predictions)
    class_labels = list(train_data.class_indices.keys())  # Map index to label
    return class_labels[class_idx]

Usage

In [28]:
print("Test directory:", test_dir)

Test directory: E:\Project_UMER\ASL_detection\ASL_detection\dataset\asl_alphabet_test
